# ABS Population Growth multi-measure

## Python set-up

In [1]:
# system imports
from pathlib import Path

# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# local imports
import plotting
from abs_data_capture import (
    AbsLandingPage,
    AbsSelectInput,
    AbsSelectionDict,
    df_from_ams,
    get_multi_series,
)
from utility import qtly_to_monthly

In [3]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
plt.style.use("fivethirtyeight")
CHART_DIR = "./CHARTS/Population/"
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)
plotting.clear_chart_dir(CHART_DIR)
plotting.set_chart_dir(CHART_DIR)
SHOW = False

## Annual growth in percentage terms

In [4]:
wanted: AbsSelectionDict = {
    "Estimated Resident Population": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="people",
            parent_topic="population",
            topic="national-state-and-territory-population",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage ERP Change Over Previous Year ;  Australia ;",
        search2="Percentage ERP Change Over Previous Year ;  Australia ;",
        abbr="--",
        calc_growth=False,
    ),
    "Civilian Population Aged 15+ years": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="labour",
            parent_topic="employment-and-unemployment",
            topic="labour-force-australia",
        ),
        table="1",
        orig_sa="Orig",
        search1="Civilian population aged 15 years and over ;  Persons ;",
        search2="Civilian population aged 15 years and over ;  Persons ;",
        abbr="--",
        calc_growth=True,
    ),
}

implicit = {
    "GDP": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="national-accounts",
            topic="australian-national-accounts-national-income-expenditure-and-product",
        ),
        table="1",
        orig_sa="Orig",
        search1="Gross domestic product: Chain volume measures ;",
        search2=" ",
        abbr="--",
        calc_growth=False,
    ),
    "GDP per capita": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="national-accounts",
            topic="australian-national-accounts-national-income-expenditure-and-product",
        ),
        table="1",
        orig_sa="Orig",
        search1="GDP per capita: Chain volume measures ;",
        search2=" ",
        abbr="--",
        calc_growth=False,
    ),
}

In [5]:
def plot_pct_growth():
    """Produce Australian population growth charts."""

    # implicit from national accounts
    i_data = get_multi_series(implicit)
    i_pop = i_data["GDP"].series / i_data["GDP per capita"].series * 1_000_000
    i_pop_growth = (
        i_pop.pct_change(
            periods=4,
            fill_method=None,
        )
        * 100
    )
    i_pop_growth = qtly_to_monthly(i_pop_growth, interpolate=False)

    # direct from ERP and labour  force
    dataset = get_multi_series(wanted)
    df = df_from_ams(dataset)
    rows = df.index.union(i_pop_growth.index)
    df = df.reindex(index=rows)
    df["Implicit population from national accounts"] = i_pop_growth

    rfooter = "ABS"
    for selector in dataset.values():
        rfooter += f" {selector.cat_id}-{selector.table}"
    rfooter += f" {list(i_data.values())[0].cat_id}-{selector.table}"

    recent = pd.Period(pd.Timestamp("today") - pd.DateOffset(years=8), freq="M")
    plotting.line_plot(
        df,
        dropna=True,
        starts=[None, recent],
        title="Population Growth (per cent per year)",
        ylabel="Per cent per year",
        lfooter="Australia. Original series. ",
        rfooter=rfooter,
        width=[3, 2, 1],
        show=SHOW,
    )


plot_pct_growth()

## Growth in number per year

In [6]:
migration_proxy = {
    "Permanent and Long-term Arrivals": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="industry",
            parent_topic="tourism-and-transport",
            topic="overseas-arrivals-and-departures-australia",
        ),
        table="1",
        orig_sa="Orig",
        search1="Number of movements ;  Permanent and Long-term Arrivals ;",
        search2="Number of movements ;  Permanent and Long-term Arrivals ;",
        abbr="--",
        calc_growth=False,
    ),
    "Permanent and Long-term Departures": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="industry",
            parent_topic="tourism-and-transport",
            topic="overseas-arrivals-and-departures-australia",
        ),
        table="2",
        orig_sa="Orig",
        search1="Number of movements ;  Permanent and Long-term Departures ;",
        search2="Number of movements ;  Permanent and Long-term Departures ;",
        abbr="--",
        calc_growth=False,
    ),
}

raw: AbsSelectionDict = {
    "Civilian Population Aged 15+ years": AbsSelectInput(
        landing_page=AbsLandingPage(
            theme="labour",
            parent_topic="employment-and-unemployment",
            topic="labour-force-australia",
        ),
        table="1",
        orig_sa="Orig",
        search1="Civilian population aged 15 years and over ;  Persons ;",
        search2="Civilian population aged 15 years and over ;  Persons ;",
        abbr="--",
        calc_growth=False,
    ),
}

In [7]:
def plot_num_growth():
    """numerical population growth charts."""

    plot = pd.DataFrame()
    sources = set()

    # civilian population
    dataset = "Civilian Population Aged 15+ years"
    raw_data = get_multi_series(raw)
    select = raw_data[dataset]
    pop = select.series.diff(12) * 1000
    plot["Civilian Population Aged 15+ years - Annual Growth"] = pop
    sources.add(f" {select.cat_id}-{select.table}")

    # implicit total population from national accounts
    datasets = ("GDP", "GDP per capita")
    i_data = get_multi_series(implicit)
    selects = [i_data[dataset] for dataset in datasets]
    i_pop = selects[0].series / selects[1].series * 1_000_000
    i_pop_growth = i_pop.diff(4)
    i_pop_growth = qtly_to_monthly(i_pop_growth, interpolate=False)
    plot["Implicit population from national accounts - Annual Growth"] = i_pop_growth
    for select in selects:
        sources.add(f" {select.cat_id}-{select.table}")

    # net migration
    datasets = (
        "Permanent and Long-term Arrivals",
        "Permanent and Long-term Departures",
    )
    mp_data = get_multi_series(migration_proxy)
    selects = [mp_data[dataset] for dataset in datasets]
    arrivals = selects[0].series.rolling(12, min_periods=12).sum()
    departures = selects[1].series.rolling(12, min_periods=12).sum()
    net_migration = arrivals - departures
    plot["Net Permanent and Long-term Migration - 12 month rolling sum"] = net_migration
    for select in selects:
        sources.add(f" {select.cat_id}-{select.table}")

    # And plot
    plot = plot / 1000
    recent = pd.Period(pd.Timestamp("today") - pd.DateOffset(years=8), freq="M")
    plotting.line_plot(
        plot,
        dropna=True,
        starts=[None, recent],
        title="Population Growth (thousand per year)",
        ylabel="Thousand per year",
        lfooter="Australia. Original series. ",
        rfooter=f"ABS:{', '.join(sorted(sources))}",
        width=[3, 2, 1],
        show=SHOW,
    )


plot_num_growth()

Caution: Could not find the 'Index' sheet in 3401013.xlsx. File not included
Caution: Could not find the 'Index' sheet in 3401014.xlsx. File not included
Caution: Could not find the 'Index' sheet in 3401015.xlsx. File not included
Caution: Could not find the 'Index' sheet in 3401016.xlsx. File not included
Caution: Could not find the 'Index' sheet in 3401017.xlsx. File not included


## Finished

In [8]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue Apr 16 2024 12:35:12

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.22.2

pandas    : 2.2.2
matplotlib: 3.8.4

Watermark: 2.4.3



In [9]:
print("Finished")

Finished
